# 3. Simple Linear Regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import janitor as jn

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

## 3.1 Simple Linear Regression

In [ ]:
# advertising = pd.read_csv('http://faculty.marshall.usc.edu/gareth-james/ISL/Advertising.csv')
advertising = pd.read_csv('../data/Advertising.csv', index_col=0)
advertising.head()

,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


### FIGURE 3.1.

In [ ]:
x = advertising.TV
y = advertising.sales
reg = stats.linregress(x, y)
y_pred = reg.intercept + reg.slope*x

In [ ]:
print(reg.intercept)
print(reg.slope)

7.032593549127702
0.0475366404330197


In [ ]:
_, ax = plt.subplots(figsize=(10, 6))

ax.scatter(x, y, color = 'red')
ax.plot(x, y_pred, color='blue', lw=0.8)

ax.set(xlabel='TV', ylabel='Sales', xlim=(-5, 305))
         
ax.vlines(
    x=advertising.TV, 
    ymin=np.fmin(advertising.sales, y_pred),
    ymax=np.fmax(advertising.sales, y_pred),
    color='k', lw=0.3)

SyntaxError: unexpected EOF while parsing (3167561280.py, line 9)

### FIGURE 3.2.

## 3.2 Multiple Linear Regression

### FIGURE 3.4.

### FIGURE 3.5.

## 3.3 Other Considerations in the Regression Model

### FIGURE 3.6.

### FIGURE 3.7.

### FIGURE 3.8.

### FIGURE 3.9.

### FIGURE 3.10.

### FIGURE 3.11.

### FIGURE 3.12.

### FIGURE 3.13.

### FIGURE 3.14.

### FIGURE 3.15.

## 3.5 Comparison of Linear Regression with K-Nearest Neighbors

## 3.6 Lab: Linear Regression

In [ ]:
from sklearn.datasets import load_boston

In [ ]:
boston_raw = load_boston()
print(boston_raw.keys())

In [ ]:
print(boston_raw.DESCR)

In [ ]:
boston_data = np.column_stack([boston_raw.data, boston_raw.target])
col_names = np.append(boston_raw.feature_names, 'MEDV')

boston = pd.DataFrame(boston_data, columns = col_names)
boston.head()

In [ ]:
boston.shape

In [ ]:
boston_clean = boston.dropna()
boston_clean.shape

Let's start by using the `statsmodels` module to fit a simple linear regression model, with `medv` as the response and `lstat` as the predictor. The basic syntax is `smf.ols('y~x', data)`, where `y` is the response, `x` the predictor, and `data` is the data frame in which these two variables are kept.

In [ ]:
lm_fit = smf.ols('MEDV~LSTAT', boston).fit()

To print some information about the model, we use the `summary()` method. This gives us p-values and standard errors for the coefficients, as well as the $R^2$ statistic and F-statistic for the model.

In [ ]:
lm_fit.summary()

We can access many of this values directly, for example, to get a list of the parameters:

In [ ]:
lm_fit.params

In order to obtain a 95% confidence interval for the coefficient estimates, we can use the `conf_int()` method.

In [ ]:
lm_fit.conf_int(alpha=0.05)

The fitted values of the model are stored in `lm_fit.fittedvalues`

The `statsmodels` module can be used to produce confidence intervals and prediction intervals for the prediction of `medv` for a given value of `lstat` (predictions).

In [ ]:
predictors = pd.DataFrame({'LSTAT': [5, 10, 15]})
predictions = lm_fit.get_prediction(predictors)
predictions.summary_frame(alpha=0.05)

For instance, the 95 % confidence interval associated with a lstat value of 10 is (24.47, 25.63), and the 95 % prediction interval is (12.828, 37.28). As expected, the confidence and prediction intervals are centered around the same point (a predicted value of 25.05 for medv when lstat equals 10), but the latter are substantially wider.

We will now plot medv and lstat along with the least squares regression line using the plot() function.

In [ ]:
_, ax = plt.subplots(figsize=(8,6))

ax.scatter(boston.LSTAT, boston.MEDV, facecolors='none', edgecolors='b', label="Data")
ax.plot(boston.LSTAT, lm_fit.fittedvalues, 'g', label="OLS")

predictions = lm_fit.get_prediction(boston).summary_frame(alpha=0.05)

ax.plot(boston.LSTAT, predictions.obs_ci_lower, color='0.75', label="Predicted Observation CI")
ax.plot(boston.LSTAT, predictions.obs_ci_upper, color='0.75', label="")

ax.plot(boston.LSTAT, predictions.mean_ci_lower, color='r',label="Predicted Mean CI")
ax.plot(boston.LSTAT, predictions.mean_ci_upper, color='r', label="")

ax.legend(loc='best')

ax.set(xlabel='LSTAT', ylabel='MEDV')

Next we examine some diagnostic plots, several of which were discussed in Section 3.3.3. Three important diagnostic plots are implemented below:

In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence

resid_t = OLSInfluence(lm_fit).resid_studentized_external

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(12,6))

axes[0].scatter(lm_fit.fittedvalues, lm_fit.resid, facecolors='none', edgecolors='b')
axes[0].set(ylabel='Residuals')

axes[1].scatter(lm_fit.fittedvalues, resid_t, facecolors='none', edgecolors='b')
axes[1].set(ylabel='Studentized Residuals')

for i in range(2):
    axes[i].set(xlabel='Fitted values')
    axes[i].axhline(y=3, color='r', linestyle='dashed')
    axes[i].axhline(y=-3, color='r', linestyle='dashed')
# |studentized residual| > 3 are generally considered outliers

In [ ]:
leverage = OLSInfluence(lm_fit).hat_matrix_diag

fig, ax = plt.subplots(figsize=(8,6))

ax.scatter(leverage, resid_t, facecolors='none', edgecolors='b')
ax.set(xlabel='Leverage', ylabel='Studentized Residuals')

To asses the normality of the residuals we can use a Normal Probability plot.

In [ ]:
from scipy.stats import probplot

_ = probplot(lm_fit.resid, plot=plt)

## Multiple Linear Regression

In order to fit a multiple linear regression model using least squares, we again use the statsmodels module. The syntax `smf.ols('y~x1+x2+x3, data)` is used to fit a model with three predictors, `x1`, `x2`, and `x3`. The `summary()` function now outputs the regression coefficients for all the predictors.

In [ ]:
lm_fit = smf.ols('MEDV~LSTAT+AGE', boston).fit()
lm_fit.summary()

The `Boston` data set contains 13 variables, and so it would be cumbersome to have to type all of these in order to perform a regression using all of the predictors. Instead, we can use the following short-hand:

In [ ]:
preds = boston.columns.tolist()
preds.remove('MEDV')
my_formula = 'MEDV~' + '+'.join(preds)
my_formula

In [ ]:
lm_ins = smf.ols(my_formula, boston)
lm_fit = lm_ins.fit()
lm_fit.summary()

We can access the individual components of a summary object by name.

In [ ]:
lm_fit.rsquared

In [ ]:
rse = np.sqrt(lm_fit.mse_resid)
rse

statsmodels can also be used to compute variance inflation factors. Most VIF’s are low to moderate for this data.

One recommendation is that if VIF is greater than 5, then the explanatory variable is highly collinear with the other explanatory variables, and the parameter estimates will have large standard errors because of this.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vifs = [(predictor, variance_inflation_factor(lm_ins.exog, idx)) 
        for (idx, predictor) in enumerate(lm_ins.exog_names)]

print('Variance Inflation Factors')
for tup in vifs:
    print('{:10}'.format(tup[0]), '{:.3f}'.format(tup[1]))

What if we would like to perform a regression using all of the variables but one? For example, in the above regression output, age has a high p-value. So we may wish to run a regression excluding this predictor. The following syntax results in a regression using all predictors except age.

In [ ]:
preds = boston.columns.tolist()
preds.remove('MEDV')
preds.remove('AGE')
my_formula = 'MEDV~' + '+'.join(preds)
my_formula

In [ ]:
lm_fit1 = smf.ols(my_formula, boston).fit()
lm_fit1.summary()

## Interaction Terms

It is easy to include interaction terms in a linear model. The syntax lstat:black tells statsmodels to include an interaction term between lstat and black. The syntax lstat*age simultaneously includes lstat, age,
and the interaction term lstat×age as predictors; it is a shorthand for lstat+age+lstat:age.

In [ ]:
smf.ols(formula='MEDV ~ LSTAT*AGE', data=boston).fit().summary()

## Non-linear Transformations of the Predictors

statsmodel can also accommodate non-linear transformations of the predictors. For instance, given a predictor $X$, we can create a predictor $X^2$ using `I(X**2)`.

In [ ]:
from statsmodels.stats.api import anova_lm

In [ ]:
lm_fit2 = smf.ols('MEDV ~ LSTAT + I(LSTAT**2)', boston).fit()
lm_fit2.summary()

The near-zero p-value associated with the quadratic term suggests that it leads to an improved model. We use the anova_lm() function to further quantify the extent to which the quadratic fit is superior to the linear fit.

In [ ]:
lm_fit = smf.ols('MEDV ~ LSTAT', boston).fit()
lm_fit2 = smf.ols('MEDV ~ LSTAT + I(LSTAT**2)', boston).fit()

anova_lm(lm_fit, lm_fit2)

Here Model 1 represents the linear submodel containing only one predictor, lstat, while Model 2 corresponds to the larger quadratic model that has two 2 predictors, lstat and lstat^2. The anova_lm() function performs a hypothesis test comparing the two models. The null hypothesis is that the two models fit the data equally well, and the alternative hypothesis is that the full model is superior. Here the F-statistic is 135 and the associated p-value is virtually zero. This provides very clear evidence that the model containing the predictors lstat and lstat2 is far superior to the model that only
contains the predictor lstat . This is not surprising, since earlier we saw evidence for non-linearity in the relationship between medv and lstat. If we plot the residuals for Model 2, there is little discernible pattern in the residuals.

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

ax.scatter(boston.LSTAT, boston.MEDV, facecolors='none', edgecolors='b', label="Data");
ax.plot(boston.LSTAT, lm_fit2.fittedvalues, 'g', marker='o',linestyle='none', label="OLS");

ax.legend(loc='best')
ax.set(xlabel='LSTAT', ylabel='MEDV')

In [ ]:
_, (ax1, ax2) = plt.subplots(1,2,figsize=(12,6))

ax1.scatter(lm_fit.fittedvalues, lm_fit.resid, facecolors='none', edgecolors='b');
ax1.set(xlabel='Fitted values', ylabel='Residuals', title='Linear Model Residuals')

ax2.scatter(lm_fit2.fittedvalues, lm_fit2.resid, facecolors='none', edgecolors='b');
ax2.set(title='Quadratic Model Residuals', xlabel='Fitted values')

In order to create a cubic fit, we can include a predictor of the form I(X**3). However, this approach can start to get cumbersome for higher-order polynomials. A better approach is the following:

In [ ]:
formula = 'MEDV ~ LSTAT +' + ' + '.join('I(LSTAT**{})'.format(i) for i in range(2, 6))
formula

In [ ]:
lm_fit5 = smf.ols(formula, boston).fit()
lm_fit5.summary()

This suggests that including additional polynomial terms, up to fifth order, leads to an improvement in the model fit! However, further investigation of the data reveals that no polynomial terms beyond fifth order have significant p-values in a regression fit.

Of course, we are in no way restricted to using polynomial transformations of the predictors. Here we try a log transformation.

In [ ]:
smf.ols('MEDV ~ np.log(LSTAT)', boston).fit().summary()

## Qualitative Predictors

We will now examine the Carseats data. We will attempt to predict Sales (child car seat sales) in 400 locations
based on a number of predictors.

In [ ]:
carseats = pd.read_csv('data/Carseats.csv', index_col = 0)
carseats.head()

The Carseats data includes qualitative predictors such as `ShelveLoc`, an indicator of the quality of the shelving location—that is, the space within a store in which the car seat is displayed—at each location. The predictor `ShelveLoc` takes on three possible values, Bad, Medium, and Good.

Given a qualitative variable such as `Shelveloc`, `statsmodels` generates dummy variables automatically. Below we fit a multiple regression model that includes some interaction terms.

In [ ]:
preds = carseats.columns.tolist()[1:]
formula ='Sales ~ ' + ' + '.join(preds) + ' + Income:Advertising + Price:Age'
formula

In [ ]:
lm_fit = smf.ols(formula, carseats).fit()
lm_fit.summary()

R has created a ShelveLoc[T.Good] dummy variable that takes on a value of 1 if the shelving location is good, and 0 otherwise. It has also created a ShelveLoc[T.Medium] dummy variable that equals 1 if the shelving location is medium, and 0 otherwise. A bad shelving location corresponds to a zero for each of the two dummy variables. The fact that the coefficient for ShelveLoc[T.Good] in the regression output is positive indicates that a good shelving location is associated with high sales (relative to a bad location). And ShelveLoc[T.Medium] has a smaller positive coefficient, indicating that a medium shelving location leads to higher sales than a bad shelving location but lower sales than a good shelving location.

# Conceptual

## 1. 

Describe the null hypotheses to which the p-values given in Table 3.4 correspond. Explain what conclusions you can draw based on these p-values. Your explanation should be phrased in terms of sales, TV, radio, and newspaper, rather than in terms of the coefficients of the linear model.

The null hypotheses are:

1) There is no relationship between amount spent on TV advertising and Sales
2) There is no relationship between amount spent on radio ads and Sales
3) There is no relationship between amount spent on newspaper ads and Sales

The p-values given in table 3.4 suggest that we can reject the null hypotheses 1 & 2, it seems likely that there is a relationship between TV ads and Sales, and radio ads and sales.

The p-value associated with the t-statistic for newspaper ads is high which suggests that we cannot reject null hypothesis 3. This suggests that there is no significant relationship between newspaper ads and sales.

## 2. 

Carefully explain the differences between the KNN classifier and KNN regression methods.

The KNN classifier determines a decision boundary which can be used to segment data into 2 or more clusters or groups. 

KNN regression is non-parmetric method for estimating a regression function that can be used to predict some quantitivie variable.

## 3. 

Suppose we have a data set with five predictors, $X_1$ = GPA, $X_2$ = IQ, $X_3$ = Gender (1 for Female and 0 for Male), $X_4$ = Interaction between GPA and IQ, and $X_5$ = Interaction between GPA and Gender. The response is starting salary after graduation (in thousands of dollars). Suppose we use least squares to fit the model, and get $β_0 = 50, β_1 = 20 , β_2 = 0.07 , β_3 = 35 , β_4 = 0.01 , β_5 = −10$ 

(a) Which answer is correct, and why?

- i. For a fixed value of IQ and GPA, males earn more on average than females.
- ii. For a fixed value of IQ and GPA, females earn more on average than males.
- iii. For a fixed value of IQ and GPA, males earn more on average than females provided that the GPA is high enough.
- iv. For a fixed value of IQ and GPA, females earn more on average than males provided that the GPA is high enough.

iii. For a fixed value of IQ and GPA, males earn more on average than females provided that the GPA is high enough.

Because X3 is our dummy variable for gender with 1 for female and 0 male, and coefficient 35, which means – all else being equal – the model will estimate a starting salary for females `$`35k higher than for males, but there is an additional interaction variable concerning GPA and gender which means if GPA > 3.5 then males earn more than females.

(b) Predict the salary of a female with IQ of 110 and a GPA of 4.0

In [ ]:
def f(gpa, iq, gender):
    return 50 + 20*gpa + 0.07*iq + 35*gender + 0.01*gpa*iq + (-10*gpa*gender)

gpa = 4
iq  = 110
gender = 1

print('$' + str(f(gpa, iq, gender) * 1000)) 

$137100.0


(c) True or false: Since the coefficient for the GPA/IQ interaction term is very small, there is very little evidence of an interaction effect. Justify your answer.

False: the interaction effect might be small but we would need to inspect the standard error to understand if this interaction effect is significant. If the standard error is also very small then it might still be considered a significant effect.

## 4. 

I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e.
$$Y = \beta_0 +\beta_1X + \beta_2X^2 +\beta_3X^3 + \epsilon$$

(a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ε. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

We would expect the training RSS for the cubic model because it is more flexible which allows it to fit more closely variance in the training data – which will reduce RSS despite this note being representative of a closer approaximation to the true linear relationship that is f(x).

(b) Answer (a) using test rather than training RSS.

We would expect the test RSS for the linear regression to be lower because the assumption of high bias is correct and so the lack of flexibility in that model is of no cost in estimating the true f(x). The cubic model is more flexible, and so is likely to overfit the training data meaning that the fit of the model will be affected by variance in the training data that is not representive of the true f(x).

(c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

We expect training RSS to decrease as the the variance/flexibility of our model increases. This holds true regardles of the true value of f(x). So we expect the cubic model to result in a lower training RSS

(d) Answer (c) using test rather than training RSS.

There is not enough information to answer this fully.

If the true relationship is highly non-linear and there is low noise (or irreducible error) in our training data then we might expect the more flexible cubic model to deliver a better test RSS.

However, if the relationship is only slightly non-linear or the noise in our training data is high then a linear model might deliver better results.

## 5. 

Consider the fitted values that result from performing linear regression without an intercept. In this setting, the i-th fitted value takes the form:

# Applied

## 8. Auto dataset for simlple linear regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
auto = pd.read_csv('../data/auto.csv')

auto = auto.drop(auto[auto.values == '?'].index)
auto = auto.reset_index()
auto.head()

,index,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [ ]:
datatypes = {'quant': ['mpg', 'cylinders', 'displacement', 'horsepower',\
                       'weight', 'acceleration', 'year'],
             'qual': ['origin', 'name']}

In [ ]:
quants = auto[datatypes['quant']].astype(np.float_)
auto = pd.concat([quants, auto[datatypes['qual']]], axis=1)

In [ ]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1,chevrolet chevelle malibu
1,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1,buick skylark 320
2,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1,plymouth satellite
3,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1,amc rebel sst
4,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1,ford torino


### (a) lm() function

Use the lm() function to perform a simple linear regression with mpg as the response and horsepower as the predictor. Use the summary() function to print the results. Comment on the output. For example:

- i. Is there a relationship between the predictor and the response?

    Yes, the low P-value associated with the t-statistic for horsepower suggests so.

In [ ]:
ax.scatter(auto['horsepower'], auto['mpg'])

NameError: name 'ax' is not defined

In [ ]:
X = auto['horsepower']
X = sm.add_constant(X)
X.head()

In [ ]:
y = auto['mpg']
lm_auto = sm.OLS(y, X)
results = lm_auto.fit()

In [ ]:
print(results.summary())

In [ ]:
ax.scatter(x=results.fittedvalues, y=results.resid)

- ii. How strong is the relationship between the predictor and the response?

    For a unit increase in horsepower, our model predicts mpg will decrease by -0.1578

- iii. Is the relationship between the predictor and the response positive or negative?

    Negative

- iv. What is the predicted mpg associated with a horsepower of 98? What are the associated 95 % confidence and prediction intervals?

In [ ]:
results.params

NameError: name 'results' is not defined

In [ ]:
x = np.array([1,98])
print("{} mpg".format(results.params.dot(x)))

In [ ]:
results.get_prediction([1,98]).summary_frame()

In [ ]:
results.conf_int()

In [ ]:
print("with 95% confidence interval [{}, {}]".format(results.conf_int().T.dot(x)[0], 
                                                     results.conf_int().T.dot(x)[1]))

In [ ]:
vifs = [(predictor, variance_inflation_factor(lm_ins.exog, idx)) 
        for (idx, predictor) in enumerate(lm_ins.exog_names)]

print('Variance Inflation Factors')
for tup in vifs:
    print('{:10}'.format(tup[0]), '{:.3f}'.format(tup[1]))

### (b) abline() function

Plot the response and the predictor. Use the abline() function to display the least squares regression line.

In [ ]:
sns.regplot(x=auto['horsepower'], y=auto['mpg'], fit_reg=True)

<AxesSubplot:xlabel='horsepower', ylabel='mpg'>

### (c) plot() function

Use the plot() function to produce diagnostic plots of the least squares regression fit. Comment on any problems you see with the fit

In [ ]:
# https://en.wikipedia.org/wiki/Studentized_residual#How_to_studentize

def lm_plots(lm, ):
    resid_t = OLSInfluence(lm).resid_studentized_external
    resid_t_sqrt = np.sqrt(np.abs(resid_t))
    leverage = OLSInfluence(lm).hat_matrix_diag
    cooks_distance = OLSInfluence(lm).cooks_distance[0]
    
    y_pred = lm.fittedvalues
    p = len(lm.params)
    
    plt.subplots(1, figsize=(16, 18))

    # Studentised residual plot
    ax1 = plt.subplot(321)
    ax1.scatter(y_pred, resid_t_sqrt, alpha=0.5, color='#3D89BE')
    sns.regplot(y_pred, resid_t_sqrt, 
                scatter=False, 
                ci=False, 
                lowess=True,
                line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    ax1.set(xlabel='Fitted values')
    ax1.set(ylabel='$\sqrt{Standardized\ residuals}}$')
    ax1.set(title='Scale-Location', fontweight='bold')
    # annotations
    abs_sq_norm_resid = np.flip(np.argsort(resid_t_sqrt), 0)
    abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]
    for i in abs_norm_resid_top_3:
        ax1.annotate(i, xy=(y_pred[i], resid_t_sqrt[i]))

    # Normal Q-Q plot
    ax2 = plt.subplot(322)
    ax2.scatter(y_pred, resid_t, alpha=0.5, color='#3D89BE')
    sns.regplot(y_pred, resid_t, 
                scatter=False, 
                ci=False, 
                lowess=True,
                line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    ax2.set(xlabel='Fitted values')
    ax2.set(ylabel='Studentised residuals')
    ax2.set(title='Externally studentised residual plot', fontweight='bold')
    # |studentized residual| > 3 are generally considered outliers
    ax2.axhline(y=3, color='r', linestyle='dashed')
    ax2.axhline(y=-3, color='r', linestyle='dashed')
    # annotations
    abs_resid = abs(lm.resid).sort_values(ascending=False)
    abs_resid_top_3 = abs_resid[:3]
    for i in abs_resid_top_3.index:
        ax2.annotate(i, xy=(y_pred[i], resid_t[i]))
    
    # Standardised Residuals
    ax3 = plt.subplot(323)
    pp = ProbPlot(resid_t)
    pp.qqplot(ax=ax2, line='45', lw=1, alpha=0.5, color='#3D89BE', )
    ax3.set(ylabel='Studentised residuals')
    ax3.set(title='Normal Q-Q', fontweight='bold')
    # annotations
    abs_norm_resid = np.flip(np.argsort(np.abs(resid_t)), 0)
    abs_norm_resid_top_3 = abs_norm_resid[:3]
    for r, i in enumerate(abs_norm_resid_top_3):
        ax3.annotate(i, xy=(np.flip(pp.theoretical_quantiles, 0)[r], resid_t[i]))

    # Residuals vs Leverage plot
    ax4 = plt.subplot(324)
    ax4.scatter(leverage, resid_t, alpha=0.5, color='#3D89BE')
    sns.regplot(leverage, resid_t, 
            scatter=False, 
            ci=False, 
            lowess=True,
            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
    ax4.set(xlabel='Leverage')
    ax4.set(ylabel='Studentised residuals')
    ax4.set(title='Externally studentised residual vs Leverage', fontweight='bold')
    # annotations
    leverage_top_3 = np.flip(np.argsort(cooks_distance), 0)[:3]
    for i in leverage_top_3:
        ax4.annotate(i, xy=(leverage[i], resid_t[i]))
    '''
    # cook's distance contours
    cooksx = np.linspace(min(leverage), max(leverage)*1.01 , 50)
    ax4.plot(cooksx, np.sqrt((0.5 * p * (1 - cooksx)) / cooksx), ls=':', c='r', label='Cook\'s distance') # 0.5 line
    ax4.plot(cooksx, np.sqrt((1 * p * (1 - cooksx)) / cooksx), ls=':', c='r') # 1 line
    ax4.legend()
    '''

In [ ]:
lm_plots(results)

NameError: name 'results' is not defined

**Non-linearity of the data**: The residual plot exhibits a discernable pattern, in this case **u-shaped**, that suggests our linear model is not providing a optimal fit to our data - the relationship is non-linear. A discernable pattern in this plot suggests that our model is failing to account for some of the reducible variance in the responses. 

**Normality**: The Q-Q plot shows that our studentised residuals have a slightly non-normal distribution.

**Heteroscedasticity** – The residual plot exhibits a conical shape. This suggests that there is some heteroscedasticity in our predictor. The standardized plot also exhibits this characteristic suggesting that standardisation doesn't alleviate the issue – to address this we might consider fitting our model by weighted least squares.

**Outliers and leverage**: The residual vs leverage plot suggests that there are several potential outliers  that could be having a strong effect (leverage) on our model. We should add more predictors to our model to clarify outliers.

**Correlation of error terms**: We see a slight increase in error above the 300th observation suggesting some correlation effect. This could mean that our estimated standard errors underestimate the true standard errors. Our confidence and prediction intervals may be narrower than they should be.

In [ ]:
def lm_residual_corr_plot(lm, y):
    y_pred = lm.fittedvalues
    resid = np.array(auto['mpg'] - y_pred)

    plt.subplots(1, figsize=(16, 5))
    ax = sns.lineplot(x=list(range(resid.shape[0])), y=resid)
    ax.set(xlabel='Observation')
    ax.set(ylabel='Residual')
    ax.set(title='Correlation of error terms', fontweight='bold')

In [ ]:
lm_residual_corr_plot(results, auto['mpg'])

NameError: name 'results' is not defined

## 9. Auto dataset for multivariate linear regression

### (a) Scatterplot matrix

Produce a scatterplot matrix which includes all of the variables in the data set.

In [ ]:
sns.pairplot(auto)

### (b) Matrix of correlations

Compute the matrix of correlations between the variables using the function cor(). You will need to exclude the name variable, which is qualitative.

In [ ]:
auto.corr()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
mpg,1.000000,-0.777618,-0.805127,-0.778427,-0.832244,0.423329,0.580541,0.565209
cylinders,-0.777618,1.000000,0.950823,0.842983,0.897527,-0.504683,-0.345647,-0.568932
displacement,-0.805127,0.950823,1.000000,0.897257,0.932994,-0.543800,-0.369855,-0.614535
horsepower,-0.778427,0.842983,0.897257,1.000000,0.864538,-0.689196,-0.416361,-0.455171
weight,-0.832244,0.897527,0.932994,0.864538,1.000000,-0.416839,-0.309120,-0.585005
acceleration,0.423329,-0.504683,-0.543800,-0.689196,-0.416839,1.000000,0.290316,0.212746
year,0.580541,-0.345647,-0.369855,-0.416361,-0.309120,0.290316,1.000000,0.181528
origin,0.565209,-0.568932,-0.614535,-0.455171,-0.585005,0.212746,0.181528,1.000000


In [ ]:
corr_matrix = auto.corr().abs()

f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corr_matrix, vmax=1, square=True)
ax.set(xticks=rotation=90)

SyntaxError: invalid syntax (651569379.py, line 5)

### (c) Multivariate linear regression

Perform a multiple linear regression with mpg as the response and all other variables except name as the predictors. Use the summary() function to print the results. Comment on the output. For instance:

- Is there a relationship between the predictors and the response?

- Which predictors appear to have a statistically significant relationship to the response?

- What does the coefficient for the year variable suggest?

In [ ]:
f = 'mpg ~ cylinders + displacement + horsepower + weight + acceleration + year + C(origin)'

model = smf.ols(f, auto).fit()
model.summary()

The coefficient for the `year` variable suggests that there is a positive relationship between `year` and `mpg` – the response variable increases as `year` increases. Specifically, this suggessts that cars manufactured more recently tend to be more fuel efficient and that all else being equal efficiency increases by 0.78 mpg/year.

In [ ]:
model.pvalues[model.pvalues < 0.05].sort_values()

In [ ]:
model.params

### (d) Outliers & High Leverage

Use the plot() function to produce diagnostic plots of the linear regression fit. Comment on any problems you see with the fit. Do the residual plots suggest any unusually large outliers? Does the leverage plot identify any observations with unusually high leverage?

In [ ]:
lm_plots(model)

**Linearity**: The u-shaped pattern in the top-left residual plot suggests that there a non-linear relationship that our model is unable to account for. The bottom left Scale-Location plot suggests that a square-root transform reduces the non-linearity of this relationship allowing our model to achieve an improved fit.

**Heteroscedasticity**: A conical pattern is apparent in the top-left plot suggesting that variance of residuals increases with response. The bottom left plot suggests that a square-root transform helps to mitigate this effect.

**Outliers**: The top-left plot suggests that there are five potential outliers towards the high end of our predicted responses. The bottom-right plot suggests that these outliers have low leverage, and so probably don't have a overly dispraportionate effect on our model.

**Leverage**: The bottom right plot reveals one point with particularly high leverage, although with a value around -1.2 this point isn't an outlier in terms of studentised residuals.

**Normality**: The top-right plot shows a normal distribution except for high value residuals. This suggests that the model under-estimates mpg for a larger number of observations than we might expect assuming a normal distribution.

### (e) The * and : symbols

Use the * and : symbols to fit linear regression models with interaction effects. Do any interactions appear to be statistically significant?

In [ ]:
f = 'mpg ~ cylinders + displacement + horsepower + weight + acceleration + year + C(origin) \
    + cylinders*displacement \
    + cylinders*horsepower \
    + cylinders*weight \
    + cylinders*acceleration \
    + cylinders*year \
    + displacement*horsepower \
    + displacement*weight \
    + displacement*acceleration \
    + displacement*year \
    + horsepower*weight \
    + horsepower*acceleration \
    + horsepower*year \
    + weight*weight \
    + weight*acceleration \
    + weight*year \
    + acceleration*year \
    '
model = smf.ols(f, auto).fit()
print(model.summary())

In [ ]:
model.pvalues[model.pvalues < 0.05].sort_values()

Notice the warning (2) that suggests the might be multicollinearity in this model. We can observe in the correlation plot above that there is strong collinearity between `mpg`, `cylinders`, `displacement`, `horespower`, and `weight`. This means further investigation is required to discern which, if any, of these features have a causal relationship with our response `mpg`.

### (f) Transformations of the variables

Try a few different transformations of the variables, such as $\log(X), \sqrt{X}, X^2$. Comment on your findings.

In [ ]:
f = 'np.log(mpg) ~ cylinders + displacement + horsepower + weight + acceleration + year + C(origin)'

model = smf.ols(f, auto).fit()
lm_plots(model)

Comment: Taking the log of the response variable dramatically improves aspects of our diagnostic plots.

- there is less pattern in the left-hand residual plots suggesting the log transform of mpg as decreased the non-linearity of its relationship to features and so improved our models ability to fit the date.

- there is a reduction in the conic shape of the bottom left plot suggesting a reduction in homoscedasticity

- the top-right plot suggests the model is now inclined to overstimate instead of underestimate

- there are no-longer any outliers exceeding a studentised residual value of three. there are now two outliers <-3

### (g) Logs of all predictors

What if we take logs of all our predictors also?

In [ ]:
f = 'np.log1p(mpg) \
    ~ np.log1p(cylinders) \
    + np.log1p(displacement) \
    + np.log1p(horsepower) \
    + np.log1p(weight) \
    + np.log1p(acceleration) \
    + np.log1p(year) \
    + C(origin) \
    + np.log1p(cylinders*displacement) \
    + np.log1p(cylinders*horsepower) \
    + np.log1p(cylinders*weight) \
    + np.log1p(cylinders*acceleration) \
    + np.log1p(cylinders*year) \
    + np.log1p(displacement*horsepower) \
    + np.log1p(displacement*weight) \
    + np.log1p(displacement*acceleration) \
    + np.log1p(displacement*year) \
    + np.log1p(horsepower*weight) \
    + np.log1p(horsepower*acceleration) \
    + np.log1p(horsepower*year) \
    + np.log1p(weight*weight) \
    + np.log1p(weight*acceleration) \
    + np.log1p(weight*year) \
    + np.log1p(acceleration*year) \
    '

model = smf.ols(f, auto).fit()
print(model.summary())

In [ ]:
model.pvalues[model.pvalues < 0.05].sort_values()

In [ ]:
lm_plots(model)

### (h) Only use the features with p-values < 0.05

What if we only use the features from this model with p-values < 0.05?

In [ ]:
import re

pat = re.compile('\[.+\]')

significant_features = model.pvalues[model.pvalues < 0.05].drop('Intercept').index
significant_features = [re.sub(pattern=pat, repl='', string=i) for i in significant_features]
list(set(significant_features))

In [ ]:
f = 'np.log(mpg) ~ ' + ' + '.join(significant_features)

model_new = smf.ols(f, auto).fit()
print(model_new.summary())

In [ ]:
lm_plots(model_new)

## 12. This problem involves simple linear regression without an intercept.

#### (a) The coefficient estimate $\hat{\beta}$

Recall that the coefficient estimate $\hat{\beta}$ for the linear regression of Y onto X without an intercept is given by (3.38). Under what circumstance is the coefficient estimate for the regression of X onto Y the same as the coefficient estimate for the regression of Y onto X?

$\hat{\beta}=1$

## 14. This problem focuses on the collinearity problem.

### (a) Perform the following commands in R:

```
set.seed(1)
x1=runif(100)
x2=0.5*x1+rnorm(100)/10
y=2+2*x1+0.3*x2+rnorm(100)
```

The last line corresponds to creating a linear model in which y is a function of x1 and x2. Write out the form of the linear model. What are the regression coefficients?

Form of the linear model is:

$\hat{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \epsilon$

Where the coefficients are:

$\beta_0 = 2$  
$\beta_1 = 2$  
$\beta_2 = 0.3$  

In [ ]:
np.random.seed(1)
x1 = np.random.uniform(size=100)
x2 = 0.5*x1 + np.random.randn(100)/10
y  = 2 + 2*x1 + 0.3*x2 + np.random.randn(100)

df = pd.DataFrame({'x1':x1, 'x2':x2, 'y':y})

### (b) Correlation between $x_1$ and $x_2$

What is the correlation between $x_1$ and $x_2$? Create a scatterplot displaying the relationship between the variables.

In [ ]:
sns.scatterplot(x='x1', y='x2', data=df);

print('Correlation coefficient: ' + str(np.corrcoef(x1, x2)[0][1]))

There is correlation between $x_1$ and $x_2$ with coefficient 0.839. The plot suggests a linear relationship with a gradient of around 0.5 and high variance.

### (c) Regression using $x_1$ and $x_2$

Using this data, fit a least squares regression to predict y using $x_1$ and $x_2$. Describe the results obtained. What are $\beta_0$, $\beta_1$, and $\beta_2$? How do these relate to the true $\beta_0$, $\beta_1$, and $\beta_2$? Can you reject the null hypothesis $H_0 : \beta_1 = 0$? How about the null hypothesis $H_0 : \beta_2 = 0$?

In [ ]:
plt.subplots(figsize=(8, 8))
ax = plt.subplot(111, projection='3d')
ax.scatter(df['x1'], df['x2'], df['y'])
ax.view_init(30, 185)
ax.set(xlabel='x1')
ax.set(ylabel='x2')
plt.show()

In [ ]:
f = 'y ~ x1 + x2'
model = smf.ols(formula=f, data=df).fit()
print(model.summary())

$\beta_0 = 2$  
$\beta_1 = 2$  
$\beta_2 = 0.3$ 

$\hat{\beta_0} = 2.1893$  
$\hat{\beta_1} = 0.7046$  
$\hat{\beta_2} = 2.5024$ 

- The intercept $\hat{\beta_0}$ is estimated fairly accurately.

- $\hat{\beta_1}$ and $\hat{\beta_2}$ are not well estimated. It's almost as though our model has got these the wrong way around.

- we cannot reject the null hypothesis for $\hat{\beta_1}$

- we can reject the null hypothesis for $\hat{\beta_2}$

### (d) Regression using $x_1$

Now fit a least squares regression to predict $y$ using only $x_1$. Comment on your results. Can you reject the null hypothesis $H_0 : \beta_1 = 0$?

In [ ]:
f = 'y ~ x1'
model = smf.ols(formula=f, data=df).fit()
print(model.summary())

In [ ]:
ax = sns.scatterplot(x='x1', y='y', data=df)

ax.set(xlabel='x1')
ax.set(ylabel='y')
ax.plot(df['x1'].sort_values(), model.predict(df['x1'].sort_values()), color='r')
ax.plot(x1, 2+(2*x1), color='y')
ax.legend(['Least squares', 'Population regression'])

### (e) Regression using $x_2$

Now fit a least squares regression to predict $y$ using only $x_2$. Comment on your results. Can you reject the null hypothesis $H_0 : \beta_1 = 0$?

In [ ]:
f = 'y ~ x2'
model = smf.ols(formula=f, data=df).fit()
print(model.summary())

In [ ]:
ax = sns.scatterplot(x='x2', y='y', data=df)
ax.set(xlabel='x2')
ax.set(ylabel='y')
ax.plot(df['x2'].sort_values(), model.predict(df['x2'].sort_values()), color='r')
ax.plot(x2, 2+(0.3*x2), color='y')
ax.legend(['Least squares', 'Population regression'])

### (f) Contradict ?

Do the results obtained in (c)~(e) contradict each other? Explain your answer.

$β_1$

- result c) suggests probability < 0.271 that the null hypothesis $H_0 :β_1 =0$ is true

- result d) suggests probability < 0.000 that the null hypothesis $H_0 :β_1 =0$ is true

$β_2$

- result c) suggests probability < 0.031 that the null hypothesis $H_0 :β_2 =0$ is true
    
- result e) suggests probability < 0.000 that the null hypothesis $H_0 :β_2 =0$ is true

Yes. We get contradictory results for the p-value associated with $\beta_1$, which in results c woul cause us to accept the null hypothesis and in d) we would reject it.

This contradiction can be explained by the collinearity of $x_1$ and $x_2$ which causes an increased standard error for all $\beta_j$ when our model incorporates both features, which in turn increases the p-values and causes us to incorrectly accept the null hypothesis $H_0 :\beta_1 =0$ when our model uses both features.

### (g) A mismeasured observation

Now suppose we obtain one additional observation, which was unfortunately mismeasured.

```
x1=c(x1, 0.1) 
x2=c(x2, 0.8)
y=c(y,6)
```
Re-fit the linear models from (c) ~ (e) using this new data. What effect does this new observation have on the each of the models? In each model, is this observation an outlier? A high-leverage point? Both? Explain your answers.

In [ ]:
df2 = df.append({'x1': 0.1, 'x2': 0.8, 'y': 6}, ignore_index=True)

model_c = smf.ols(formula='y ~ x1 + x2', data=df2).fit()
model_d = smf.ols(formula='y ~ x1', data=df2).fit()
model_e = smf.ols(formula='y ~ x2', data=df2).fit()

- The addition of this anomaly has increased model c's p-value for $\hat{\beta_1}$, suggesting even more strongly the false acceptance of the null hpyothesis.

- When considered seperately the estimates for $\hat{\beta_1}$ and $\hat{\beta_2}$ are worsened by introducing this anomaly.

In [ ]:
lm_plots(model_c)

In [ ]:
lm_plots(model_d)

In [ ]:
lm_plots(model_e)

- model_c : the anomaly is not an outlier and very high leverage

- model_d : the anomaly is an outlier but not high leverage

- model_e : the anomaly is not an outlier but fairly high leverage

## 15. Boston data set

### (a) Simple linear regression

For each predictor, fit a simple linear regression model to predict the response. Describe your results. In which of the models is there a statistically significant association between the predictor and the response? Create some plots to back up your assertions.

In [ ]:
from sklearn.datasets import load_boston

In [ ]:
boston = load_boston()
boston = pd.DataFrame(boston.data, columns=boston.feature_names)

assert boston.isnull().sum().sum() == 0

models_a = [smf.ols(formula='CRIM ~ {}'.format(f), data=boston).fit()
            for f in boston.columns[1:]]

for model in models_a:
    dp = 4
    display(pd.DataFrame({'coef':    np.around(model.params, dp),
                          'std_err': np.around(model.bse, dp),
                          't':       np.around(model.params / model.bse, dp),
                          'P>|t|':   np.around(model.pvalues, dp),
                          '[0.025':  np.around(model.conf_int(alpha=0.05)[0], dp),
                          '0.975]':  np.around(model.conf_int(alpha=0.05)[1], dp)}))

In [ ]:
from IPython.display import display, HTML

display(HTML('<h4>p < 0.05</h4>'))
for model in models_a:
    if model.pvalues[1] < 0.05:
        print(model.params[1:].index[0])

display(HTML('<h4>p > 0.05</h4>'))
for model in models_a:
    if model.pvalues[1] > 0.05:
        print(model.params[1:].index[0])

In [ ]:
def plot_grid(df, response, cols):
    variables = df.columns.drop(response)
    for i in range(0, len(variables), cols):
        g = sns.PairGrid(df, y_vars=[response], x_vars=variables[i:i+cols], height=4)
        g.map(sns.regplot);
    return

plot_grid(boston, 'CRIM', 4)

### (b) Multivariate regression

Fit a multivariate regression model to predict the response using all of the predictors. Describe your results. For which predictors can we reject the null hypothesis $H_0 : \beta_j = 0$?

In [ ]:
response   = 'CRIM'
predictors = boston.columns.drop(response)
f = '{} ~ {}'.format(response, '+'.join(predictors))

model_b = smf.ols(formula=f, data=boston).fit()
model_b.summary()

In [ ]:
display(HTML('<h4>p < 0.05</h4>'))

model_b.pvalues[model_b.pvalues < 0.05]

### (c) Comparison

How do your results from (a) compare to your results from (b)? Create a plot displaying the univariate regression coefficients from (a) on the x-axis, and the multiple regression coefficients from (b) on the y-axis. That is, each predictor is displayed as a single point in the plot. Its coefficient in a simple linear regression model is shown on the x-axis, and its coefficient estimate in the multiple linear regression model is shown on the y-axis.

Multivariate regression found 4 of 12 predictors to be significnat where univariate regression found 11 of 12 significant. Multivariate regression seems to find significanlty less predictors to be significant.

In [ ]:
univariate_params = pd.concat([m.params[1:] for m in models_a])
multivariate_params = model_b.params[1:]

df = pd.DataFrame({'Univariate_coef': univariate_params, 
                   'Multivariate_coef': multivariate_params,})
display(df)

plt.subplots(figsize=(16, 8))
ax = sns.scatterplot(x='Univariate_coef', y='Multivariate_coef', data=df)

NameError: name 'models_a' is not defined

### (d) Non-linear association

Is there evidence of non-linear association between any of the predictors and the response? To answer this question, for each predictor X, fit a model of the form

$Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \epsilon$

In [ ]:
models_d = [smf.ols(formula='CRIM ~ {0} + np.power({0}, 2) + np.power({0}, 3)'.format(f), data=boston).fit()
            for f in boston.columns[1:]]

for model in models_d:
    dp = 4
    display(pd.DataFrame({'coef':    np.around(model.params, dp),
                          'std_err': np.around(model.bse, dp),
                          't':       np.around(model.params / model.bse, dp),
                          'P>|t|':   np.around(model.pvalues, dp),
                          '[0.025':  np.around(model.conf_int(alpha=0.05)[0], dp),
                          '0.975]':  np.around(model.conf_int(alpha=0.05)[1], dp)}))

NameError: name 'boston' is not defined

In [ ]:
# In which of the models is there a statistically significant association between the predictor and the response?

display(HTML('<h4>Features with p < 0.05</h4>'))

sig = pd.concat([model.pvalues[model.pvalues < 0.05] for model in models_d])

display(pd.DataFrame({'P>|t|':sig.drop('Intercept')}))

- There is evidence of a non-linear association between the repsonse and INDUS, NOX, AGE, DIS, PTRATIO

- There is evidence of a linear association between the response and ZN